![](https://matthewmazur.files.wordpress.com/2018/03/neural_network-9.png)

In [46]:
import numpy as np
def logistic(val):
	return 1 / (1 + np.e ** (-val) )

def logistic_partial(x):
	return x * (1-x)

In [21]:
x = [0.05, 0.1]
y = [0.01, 0.99]
w = [0.15, 0.20, 0.25, 0.30, 0.40, 0.45, 0.50, 0.55]
b = [0.35, 0.60]

In [36]:
def calc_forward_pass(x, w, b, activation_function):
	out = 0
	for i in range(len(x)):
		out += w[i] * x[i]

	out += b

	return out, activation_function(out)

def split(x):
	net = [e[0] for e in x]
	out = [e[1] for e in x]

	return net, out

In [39]:
net_h1, out_h1 = split([
	calc_forward_pass(x, w[0:2], b[0], logistic),
	calc_forward_pass(x, w[2:4], b[0], logistic)
])
net_o, out_o = split([
	calc_forward_pass(out_h1, w[4:6], b[1], logistic),
	calc_forward_pass(out_h1, w[6:8], b[1], logistic)
])
print ("layer 1 output values", out_h1)
print ("output values", out_o)

layer 1 output values [0.5932699921071872, 0.596884378259767]
output values [0.7513650695523157, 0.7729284653214625]


In [75]:
# Calculate E Total
def mse(y, y_pred):
	return 1/2 * (y - y_pred) ** 2
Error_o = [mse(y[i], out_o[i]) for i in range(len(y))]
E_total = sum(Error_o)

print ("MSE output errors", Error_o)
print ("Total Error", E_total)

MSE output errors [0.274811083176155, 0.023560025583847746]
Total Error 0.2983711087600027


In [63]:
def delta_rule(target, out, input, partial_activation):
	res = -(target - out) 
	res *= partial_activation(out)
	res *= input

	return res

In [79]:
# Apply backwards pass
E_total_per_out = [
	-(y[i] - out_o[i]) for i in range(len(y))
]
out_per_net = [
	logistic_partial(out) for out in out_o
]
net_per_weight = out_h1


print ("E_total_per_out", E_total_per_out)
print ("out_per_net", out_per_net)
print ("net_per_weight", net_per_weight)


res = E_total_per_out[0] * out_per_net[0] * net_per_weight[0]
print ("total", res)
print (delta_rule(
	y[0],
	out_o[0],
	out_h1[0],
	logistic_partial
	))

print (-(y[0] - out_o[0]) * logistic_partial(out_o[0]) * out_h1[0])

E_total_per_out [0.7413650695523157, -0.21707153467853746]
out_per_net [0.18681560180895948, 0.17551005281727122]
net_per_weight [0.5932699921071872, 0.596884378259767]
total 0.08216704056423078
0.08216704056423078
0.08216704056423078


In [80]:
# Apply backwards pass
alpha = 0.5		# learning rate
res = delta_rule(
	y[0],
	out_o[0],
	out_h1[0],
	logistic_partial)

def calc_new_weight(w, influence):
	return w - alpha * influence


new_weight = calc_new_weight(w[5], res)
print ("total", res)
print ("new weight", new_weight)


total 0.08216704056423078
new weight 0.40891647971788464


In [81]:
new_w6  = calc_new_weight(w[5], delta_rule(
	y[0],
	out_o[0],
	out_h1[1],
	logistic_partial
))

new_w7  = calc_new_weight(w[6], delta_rule(
	y[1],
	out_o[1],
	out_h1[0],
	logistic_partial
))

new_w8  = calc_new_weight(w[7], delta_rule(
	y[1],
	out_o[1],
	out_h1[1],
	logistic_partial
))


print ("new weights 6-8", new_w6, new_w7, new_w8)

new weights 6-8 0.4086661860762334 0.5113012702387375 0.5613701211079891


In [91]:
def calc_E_output__out_hidden(out_index, net_index, weight):
	return E_total_per_out[out_index] * out_per_net[net_index] * weight

E_output__out_hidden = [
	calc_E_output__out_hidden(0, 0, w[4]),
	calc_E_output__out_hidden(1, 0, w[5])
]

print (E_output__out_hidden)

[0.05539942465142279, -0.018248557223954427]
